<div align = "center">

# **Menyimpan dan Memuat Model serta Checkpoint**

</div>

> Pada tutorial ini akan dibahas secara komprehensif cara-cara untuk menyimpan dan memuat model, serta menyimpan checkpoint ketika melakukan training. Juga akan dibahas cara-cara untuk menyimpan parameter secara online menggunakan wandb.

### Mengimpor library

In [ ]:
import torch
import torch.nn as nn

---
## **Menyimpan Model** (Metode Sederhana)

Misalnya kita memiliki model sebagai berikut

In [ ]:
class ContohModel(nn.Module):
    def __init__(self, n_input):
        super().__init__()
        self.linear = nn.Linear(n_input, 1)

    def forward(self, x):
        preds = torch.sigmoid(self.linear(x))
        return preds

model = ContohModel(n_input=6)

Anggaplah model diatas sudah dilatih. Untuk menyimpan model ini secara sederhana, dapat dilakukan dengan:

```python
torch.save(model, FILE)
```

Apabila model ingin disimpan di direktori lain, silahkan masukkan path pada `FILE`

In [ ]:
NAMAFILE = 'contohmodel.pth'
torch.save(model, NAMAFILE)

**Memuat Model** (Metode Sederhana)
Untuk mencetak setiap parameter di model, dapat dilakukan dengan:

```python
for param in model.parameters():
    print(param)
```


In [ ]:
model = torch.load(NAMAFILE)
model.eval()

print(model)

for param in model.parameters():
    print(param)

# dan dilanjutkan dengan proses evaluasi

---
## **Menyimpan Model** Dengan `State_Dict`

Cara menyimpannya adalah sebagai berikut:

In [ ]:
NAMAFILE2 = 'contohmodel2.pth'
torch.save(model.state_dict(), NAMAFILE2)

Sementara itu cara memuatnya agak sedikit berbeda. Dengan cara ini yang disimpan adalah sebuah dictionary yang berisi informasi model.

In [ ]:
model2 = ContohModel(n_input=6)
model_yangdimuat = torch.load(NAMAFILE2)
model2.load_state_dict(model_yangdimuat)
print(model2)

model2.eval()

for param in model2.parameters():
    print(param)

> Penjelasan: Apa yang dimaksud dengan state_dict?

State_dict menyimpan nilai dari parameter dari setiap layer yang ada pada model. Pada model _dummy_ yang kita buat diatas hanya terdapat satu layer berupa linear layer yang memiliki parameter berupa weight dan bias. Dikarenakan kita menggunakan 6 input, maka parameter yang tersimpan adalah sebuah tensor berisi enam buah elemen weight beserta elemen bias. Mari kita coba print state_dict untuk melihat elemen tersebut.

In [ ]:
print(model2.state_dict())

Bahkan, state_dict juga menyimpan parameter untuk optimizer

In [ ]:
l_rate = 0.01
optimizer = torch.optim.Adam(model2.parameters(), lr=l_rate)
print(optimizer.state_dict())

---
## **Checkpoint**

Menyimpan dan memuat model checkpoint dilakukan supaya model dapat dikembalikan ke state yang sama ketika melakukan training. Kasus penggunaan checkpoint yang kerap ditemukan adalah ketika melanjutkan proses training yang terhenti. Saat menyimpan pos pemeriksaan umum, item yang harus disimpan lebih dari sekadar state_dict model.

In [ ]:
checkpoint = {
    "epoch": 90,
    "model_state": model2.state_dict(),
    "optim_state": optimizer.state_dict(),
}

torch.save(checkpoint, 'checkpoint.pth')

In [ ]:
load_ckpth = torch.load('checkpoint.pth')
epoch = load_ckpth['epoch']

# memuat kembali model dan optimizer
model3 = ContohModel(n_input=6)
optimizer = torch.optim.Adam(model3.parameters(), lr=0) # lr dapat dikosongkan dan dimuat kembali kemudian

model3.load_state_dict(load_ckpth['model_state'])
optimizer.load_state_dict(load_ckpth['optim_state'])

print(model3.state_dict())
print(optimizer.state_dict())


---
## **Menyimpan dari dan ke device yang berbeda**

Jika anda memiliki model yang berada di device yang berbeda, maka ada sedikit hal yang harus dilakukan ketika menyimpan state_dict dari model

Contoh menyimpan model dari GPU dan memuatnya di CPU

In [ ]:
# Model yang di train pada GPU, dimuat di CPU

device = torch.device("cuda")
model.to(device)
## anggap model sudah dilatih
torch.save(model.state_dict(), 'model_gpu.pth')

# cara memuat
device = torch.device("cpu")
model = ContohModel(n_input=6)
model.load_state_dict(torch.load('model_gpu.pth', map_location=device))

In [ ]:
device = torch.device("cpu")
model = ContohModel(n_input=9)
torch.save(model.state_dict(), 'model_cpu.pth')

# cara memuat
device = torch.device("cuda")
model = ContohModel(n_input=9)
model.load_state_dict(torch.load('model_cpu.pth', map_location="cuda:0"))
model.to(device)

---
# Referensi
[Pytorch Saving and Loading Checkpoint](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html)